<a href="https://colab.research.google.com/github/TeraLoi/python/blob/master/OneginBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Превращаем «Евгения Онегина» в чат-бот

Чат-боты — один из самых популярных трендов последних лет. И неудивительно: мы не расстаемся со смартфонами, при этом большая часть экранного времени уходит на соцсети и мессенджеры. Это значит, что общаться с клиентами и пользователями удобнее всего именно там. 

Чтобы создавать чат-ботов с нуля, нужно уметь программировать — например, на Python. Однако с помощью этого тьюториала вы сможете самостотельно запустить чат-бота в Telegram без каких бы то ни было навыков программирования и даже не устанавливая Python на компьютер!



## Шаг 1. Устанавливаем библиотеку для работы с Telegram API

Как и у любого уважающего себя сервиса, у телеграма есть API *(application program interface)* — специальный набор инструментов для взаимодействия с программой, а не с живым пользователем. Вот [официальная страница](https://core.telegram.org/bots/api) с  его документацией. 

Для популярных API обычно можно найти готовые «обертки» на разных языках программирования. Они облегчают работу с API, позволяя не обращаться к нему напрямую. Существует довольно много библиотек для работы с Telegram API на Python:
* [telepot](https://github.com/nickoala/telepot) 
* [python-telegram-bot](https://github.com/python-telegram-bot/python-telegram-bot)
* [pyTelegramBotAPI](https://github.com/eternnoir/pyTelegramBotAPI). 

Мы будем использовать последнюю из них — `pyTelegramBotAPI`. Чтобы ее установить, нужно просто запустить следующую ячейку, нажав на кнопочку ▶️ слева от нее или `Ctrl+Enter`. Когда на месте ▶️ появится цифра — значит, код успешно выполнен.

In [ ]:
!pip install pyTelegramBotAPI

## Шаг 2. Получаем API Token

Чтобы создать бота, нам необходимо получить уникальный идентификатор для доступа к Telegram API, который называется **токеном**. Как это сделать?

1. Если нет аккаунта в телеграме, то нужно зарегистрироваться — https://web.telegram.org (ввести номер телефона, получить смс с кодом и ввести код на странице подтверждения номера).

2. Следующий шаг — начать чат с крестным отцом всех ботов, https://t.me/BotFather (просто кликните по этой ссылке или наберите в поиске телеграма @BotFather). Далее,

  * отправляем ему команду `/newbot`;
  * вводим имя бота — то, что будет отображаться как имя всем, кто будет общаться с нашим ботом;
  * вводим юзернейм бота — он должен заканчиваться на `bot` или `Bot`. Если введенный юзернейм уже занят, то `BotFather` предложит придумать другой.

После этого вы получите API токен, уникальный для вашего бота. Если у вас несколько ботов, то у каждого токен будет свой!

<img src="https://raw.githubusercontent.com/ancatmara/skillfactory/master/img/Inkedget_token_LI.jpg" width="550" align="left">

Теперь нужно скопировать полученный токен и сохранить его в отдельном файле. Для этого просто запустите следующую ячейку и вставьте в открывшуюся строку ваш токен, а затем нажмите `Enter`.

In [ ]:
with open("conf.py", "w", encoding="utf-8") as f:
    f.write('TOKEN = "%s"' % input("Введите ваш токен"))

## Шаг 3. Загружаем данные

Отлично, теперь можно переходить к самому интересному — к написанию логики бота! Допустим, мы хотим, чтобы он умел:

1. Присылать пользователю случайные цитаты из определенного текста;
2. Присылать цитаты с заданным словом.

В качестве примера возьмем «Евгения Онегина» — уж у Пушкина-то точно найдутся строчки на все случаи жизни! А вообще можете использовать любые данные, которые вам нравятся — от текстов песен любимой группы до сборника латинских афоризмов. Главное условие, чтобы это был файл формата `.txt` с кодировкой `utf-8`.

Чтобы скачать текст «Евгения Онегина», запустите следующую ячейку.

In [ ]:
!wget https://raw.githubusercontent.com/ancatmara/skillfactory/master/data/onegin.txt

Если же вы хотите использовать другой текст, то нажмите на значок 📁 слева, выберите `Upload` и загрузите нужный txt-файл cо своего компьютера.

<img src="https://raw.githubusercontent.com/ancatmara/skillfactory/master/img/upload.png" width="350" align="left">

## Шаг 4. Пишем код бота на Python

А теперь переходим непосредственно к коду бота на Python! Если вы совсем не знакомы с программированием, то не стоит бояться — вам предстоит просто запускать ячейки с кодом, как вы уже делали выше 🙂. 

Для начала импортируем нужные библиотеки и напишем две вспомогательные функции:
1. `get_sentences` для разбиения текста из файла на предложения;
2. `random_quote` для выбора случайного предложения.

**Функция** — это описание того, что нужно сделать с данными, а **библиотека** — готовый набор разных функций.

In [ ]:
import re
import conf
import random
import telebot

def random_quote(sents):
    i = random.randint(0, len(sents)-1)
    return sents[i]

def get_sentences(path, regex=".*?[.!?]\s"):
    s = re.compile(regex, re.DOTALL)
    with open(path, "r", encoding="utf-8") as f:
        sents = s.findall(f.read())
    return sents

Прочитаем текст и разобьем его на предложения. Если вы загрузили свой файл, то измените `onegin.txt` на его название.

In [ ]:
sents = get_sentences("onegin.txt")

Создадим бота, используя наш токен.

In [ ]:
bot = telebot.TeleBot(conf.TOKEN)

Теперь напишем **обработчики сообщений** *(message handlers)* — специальные функции, которые позволят боту реагировать на сообщения. В общем случае обработчик сообщений выглядит так:

```python
# указываем, что это обработчик сообщений
@bot.message_handler(...)
# определение функции
def my_function(message):
    # код, который генерирует ответ
    reply = 'Привет!'
    # код для отправки сообщения в чат
    bot.send_message(message.chat.id, reply)
```

* `bot` — это имя переменной с нашим ботом;
* `...` — фильтр (то, на какие сообщения будет реагировать функция)

Подробнее про обработчики сообщений и фильтры можно почитать в [документации библиотеки](https://github.com/eternnoir/pyTelegramBotAPI#message-handlers).

Для начала напишем функцию `send_quote`, которая по команде `\send` будет отправлять пользователю случайное предложение из текста.

In [ ]:
@bot.message_handler(commands=['send'])
def send_quote(message):
    bot.send_message(message.chat.id, random_quote(sents))

Обработчик сообщений необязательно связан с какой-то командой. Например, можно написать такую функцию, которая будет реагировать на любое сообщение. Воспользуемся этим, чтобы реализовать поиск по тексту — примерно как `Ctrl+F/Сmd+F` в текстовом редакторе или браузере. 

То есть, если пользователь отправит боту какое-то слово или фразу, мы будем искать точное совпадение в «Евгении Онегине» и отправим пользователю результат. Если найдется несколько предложений с заданным словом, то случайным образом выберем одно из них; если же не найдется ничего, честно скажем об этом пользователю.

In [ ]:
@bot.message_handler(func=lambda m: True)
def query_quote(message):
    query = message.text
    res = [sent for sent in sents if query.lower() in sent.lower()]
    if len(res) != 0:
        bot.send_message(message.chat.id, random_quote(res))
    else:
        bot.send_message(message.chat.id, "Об этом Пушкин не писал!")

Теперь сделаем так, чтобы наш бот постоянно спрашивал сервера телеграма, не пришло ли ему новых сообщений. Параметр `none_stop=True` означает, что при возникновении ошибок бот должен стараться не прекращать работу.

```python
if __name__ == '__main__':
    bot.polling(none_stop=True)
```

## Шаг 5. Запускаем бота

Соберем весь код в одной ячейке. Чтобы бот заработал, нужно просто запустить ее! Теперь нашего бота можно найти в телеграме и написать ему.

Обратите внимание, что бот будет работать, пока запущена ячейка — т.е. сейчас не нужно дожидаться, пока на месте ▶️ появится цифра. Когда вы захотите остановить бота, нужно нажать ⏹. Впрочем, даже вы этого не сделаете, то после некоторого времени бездействия Google Colab сам остановит процесс.

In [ ]:
import re
import conf
import random
import telebot

def get_sentences(path, regex=".*?[.!?]\s"):
    with open(path, "r", encoding="utf-8") as f:
        sents = re.findall(regex, f.read(), re.DOTALL)
    return sents

def random_quote(sents):
    i = random.randint(0, len(sents)-1)
    return sents[i]

bot = telebot.TeleBot(conf.TOKEN) 
sents = get_sentences("onegin.txt")

@bot.message_handler(commands=['send'])
def send_quote(message):
    bot.send_message(message.chat.id, random_quote(sents))
    
@bot.message_handler(func=lambda m: True) 
def query_quote(message):
    query = message.text
    res = [sent for sent in sents if query.lower() in sent.lower()]
    if len(res) != 0:
        bot.send_message(message.chat.id, random_quote(res))
    else:
        bot.send_message(message.chat.id, "Об этом Пушкин не писал!")

if __name__ == '__main__':
    bot.polling(none_stop=True)

## Шаг 6. Тестируем бота

Посмотрим, что может наш бот. Найти его в телеграме можно по тому юзернейму, который вы указали — например, `@EugeneOneginBot`.

1. Если отправить ему команду `/send`, то он пришлет случайную цитату (одно предложение) из «Евгения Онегина».

<img src="https://raw.githubusercontent.com/ancatmara/skillfactory/master/img/send.png" width="550" align="left">




2. Если отправить ему любое слово или фразу, он попытается найти ее в тексте и прислать нам результат.

<img src="https://raw.githubusercontent.com/ancatmara/skillfactory/master/img/search.png" width="550" align="left">

3. Если ничего не найдется, бот нам об этом скажет.

<img src="https://raw.githubusercontent.com/ancatmara/skillfactory/master/img/not_found.png" width="550" align="left">

## Шаг 7. Дополнительные настройки

С помощью `@BotFather` можно сделать для нашего бота описание, список команд, аватарку и еще много чего.

<img src="https://raw.githubusercontent.com/ancatmara/skillfactory/master/img/botfather.png" width="350" align="left">



**Список команд**

<img src="https://raw.githubusercontent.com/ancatmara/skillfactory/master/img/setcommands.png" width="500" align="left">

Теперь, когда пользователь введет `\` в чате с ботом, он увидит выплывающий список команд:

<img src="https://raw.githubusercontent.com/ancatmara/skillfactory/master/img/commands.png" width="500" align="left">

**Описание**

<img src="https://raw.githubusercontent.com/ancatmara/skillfactory/master/img/setdescription.png" width="500" align="left">

Описание появится в самом начале чата с ботом.

<img src="https://raw.githubusercontent.com/ancatmara/skillfactory/master/img/descr.png" width="500" align="left">

Поздравляем, вы справились! Как видите, в программировании ничего сложного нет. 

Эта задача — ваш первый маленький, но важный шаг на пути к серьезной профессии. Стать профессионалом и получить высокооплачиваемую профессию можно с нуля. 

Больше секретов по ссылке: https://clck.ru/P5ACf
